### Setup the environment
https://joelmccune.com/anaconda-and-arcgis-pro/
create a file named `enviroment.yml` containing the following text

```
# https://joelmccune.com/anaconda-and-arcgis-pro/
name: arcgis

channels:
  - esri 
  - conda-forge
  - defaults
  
dependencies:
  - arcgis
  - arcpy
  - deep-learning-essentials
  - cookiecutter
  - nodejs
  - python=3.7
```

open command prompt as administrator
navigate to directory where environment.yml is saved

```
> conda env create -f [C:\path\to\your\]environment.yml
> conda activate arcgis
```

updating the environment, edit the enviromnent.yml file then navigate to the directory and enter the following command in the terminal as administrator. 

```
conda env update --prefix arcgis --file environment.yml  --prune
```

### Preprpcessing 

1. Union > Dissolve Drago 500yr floodplains
2. Project Trueheart to Drago, 
3. Union Trueheart and Drago
4. Dissolve Imperv > calculate field Imperv = 1, Feature to Raster

In [42]:
# Reset environment settings to default settings.
arcpy.ResetEnvironments()

# setup work space
import arcpy
import os
import glob
import tempfile
# Set environment settings
arcpy.env.workspace = "C:/Workspace/VTWetlands/VTWetlands/"
wd = arcpy.env.workspace + "/"
print(os.listdir(wd))
print(arcpy.env.workspace)
outfolder = arcpy.env.workspace + "/outputs/"
print(outfolder)

# random number generator and seed 
arcpy.env.randomGenerator = "99 ACM599"

# set the number of cores to distribute processing to
arcpy.env.parallelProcessingFactor = "50%"

# output coordinate system
template_SRS = r"D:\GeoData\LandLandcov_BaseLC2016\LandLandcov_BaseLC2016.tif"
spatial_ref = arcpy.Describe(template_SRS).spatialReference
arcpy.env.outputCoordinateSystem = spatial_ref 

import pandas as pd 
import numpy as np
from shapely.geometry import Point
from arcgis.geometry import Geometry
import math


# read in list of selected sites and shuffle values
df = pd.read_csv(wd+"sites/selected_sites_2021-03-04.csv")
roster = pd.read_csv(wd+"sites/roster.csv")
netid = roster.NETID
nstudents = len(netid)
print(netid)
print(df)
nreps = 7


['.backups', '.ipynb_checkpoints', '.pyHistory', 'AssignRandomPoints2KMLs.ipynb', 'backup.aprx', 'ImportLog', 'Index', 'LandCoverClassification_SamplingPlots.gdb', 'LandCoverClassification_SamplingPlots.lyrx', 'LandLandcov_ImpervSrfcs2016', 'Map1.pdf', 'Map1_CDL19.pdf', 'Map1_Imagery.pdf', 'Map_image.pdf', 'Map_wetland.pdf', 'New Notebook.ipynb', 'outputs', 'sites', 'temp', 'VSWI_Wetlands_Advisory_Layer', 'VSWI_Wetlands_Advisory_Layer.zip', 'VSWI_Wetlands_Class_Layer', 'VSWI_Wetlands_Class_Layer.zip', 'VTWetlands.aprx', 'VTWetlands.aprx.xml', 'VTWetlands.gdb', 'VTWetlands.tbx', 'VTWetlands_2021-03-04.aprx', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.zip']
C:/Workspace/VTWetlands/VTWetlands
C:/Workspace/VTWetlands/VTWetlands/outputs/
0      sralger
1     bbalpard
2     nebingha
3     sbradle2
4     lbrown28
        ...   
72      extra1
73      extra2
74      extra3
75      extra4
76      extra5

In [43]:
# define functions
def random_selection1(inlayer,nstudents,IDmin,z,npoints,col,netid,i):
    y = math.ceil(npoints*z)
    IDmax= npoints+ IDmin
    files = [None]*nstudents
    for x in range(nstudents): #for loop
        if x>0:
            IDmin=y+IDmin
            IDmax=IDmin+npoints
        print("index", x)
        print("student", netid[x])
        print(IDmin, IDmax)
        # set sql string
        sql=col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)
        
        if x==nstudents : #for the last student wrap to the first student
            IDmax= IDmin + y
            sqllow=col+" > " +"0"+"  And "+col+" <= "+str(y)
            sqlhigh=" OR "+col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)
            sql=sqllow + sqlhigh
        print(sql)
        # select by attribute 
        
        selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
        fname = "points"+str(i)+"_Brandom_"+netid[x]
        files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
        #arcpy.conversion.LayerToKML(layer=selected,
        #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
        arcpy.management.Delete(selected)
    return(IDmin)

def sample_selection2(inlayer,nstudents,IDmin,z,npoints,col,netid,i,arr,nstudentsperpoint):
    IDmax= IDmin + npoints
    files = [None]*nstudents
    print(files)
    for x in range(nstudents): #for loop
        w = math.floor(x/nstudentsperpoint)
        if x == 0: wprior = w
        if w > wprior:
            wprior = w
            IDmin = IDmin+1
        IDmax = IDmin+npoints
            
        rands = arr[IDmin:IDmax]
        sql = " OR ".join(["%s = %s" % (col,str(r)) for r in rands])
        print("index", x)
        print("student", netid[x])
        print(w,rands)
        print(sql)
        # select by attribute 
        selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
        fname = "points"+str(i)+"_Asample_"+netid[x]
        files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
        #arcpy.conversion.LayerToKML(layer=selected,
        #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
        arcpy.management.Delete(selected)
    return(IDmax)

def globsearchinpath(inpath,globsearch):
    os.chdir(inpath)
    return(glob.glob(globsearch))

def os_cmd_ogr2ogr (inpath, outpath, outformat):
    cmd = "ogr2ogr -f \"{outformat}\" {outpath} {inpath}".format(outformat=outformat,
                                                                 outpath=outpath,
                                                                 inpath=inpath)
    print(cmd)
    try:
        os.system(cmd)
        print("success")
    except:
        print("error")
    return()

# extract values from list of rasters at points
def arcpy_globExtractMultiValues2Points(inpath,globsearch,inpoints):
    os.chdir(inpath)
    inrasters = glob.glob(globsearch)
    print(inrasters)
    return(arcpy.sa.ExtractMultiValuesToPoints(inpoints,inrasters))

## Prepare Floodplain Polygon Layer

In [44]:
# clip RCPP sites to 
infile = "Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017/Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.shp"
clipfile = r"C:\Workspace\VTWetlands\VTWetlands\VTWetlands.gdb\Floodplain_500yr_drago_Proje"
clipped = outfolder+"RCCP_Floodplain_Clip.shp"
arcpy.analysis.Clip(infile, clipfile, clipped)

# dissolve polygons of constraining feature
inpath = clipped
print(inpath)
outfeatures = "Dissolved.shp"
outpath = outfolder + outfeatures
# check if file exists and delete before running script

if os.path.exists(outpath):
    arcpy.management.Delete(outpath)
if os.path.exists(inpath):
    arcpy.Dissolve_management (inpath, outpath)
else: 
    print("input file does not exist")


C:/Workspace/VTWetlands/VTWetlands/outputs/RCCP_Floodplain_Clip.shp


## Generate Random Points and Create Unique IDs

In [45]:
# create n random points inside one dissovled polygon
outname = "dissolved_randompoints.shp"
npoints = nstudents * 7 * 10
if os.path.exists(outfolder+outname):
    print("deleting ",outfolder+outname)
    arcpy.management.Delete(outfolder+outname)
random = arcpy.management.CreateRandomPoints(out_path=outfolder, 
                                    out_name=outname, 
                                    constraining_feature_class=outpath,             
                                    number_of_points_or_field=npoints)

random = arcpy.management.CopyFeatures(in_features=random,
                                       out_feature_class=wd+"VTWetlands.gdb/dissolved_random")

# create new field name for random points

codeblock = '''
def concat(x):
    return('r'+str(x))
    '''
expression = "concat(!OBJECTID!)"
inTable = random
fieldName = "Name"
random = arcpy.management.CalculateField(inTable, fieldName, expression, "PYTHON3", 
                                codeblock)
fieldList = arcpy.ListFields(random)  #get a list of fields for each feature class
print([fn.name  for fn in fieldList])

# create new field name for selected points

codeblock = '''
def concat(x):
    return('v'+str(x))
    '''
expression = "concat(!IDN!)"
selected = wd+"sites/mygeodata_csv_shp/selected_sites_2021-03-04-point.shp"
#newfeature = "copy_selected_sites"
fieldName = "Name"
selected = arcpy.management.CopyFeatures(in_features=selected,
                                         out_feature_class=wd+"VTWetlands.gdb/copy_selected_sites")
fieldList = arcpy.ListFields(selected)  #get a list of fields for each feature class
print([fn.name  for fn in fieldList])
"""for field in fieldList: #loop through each field
    if field.name.lower() == 'name':  #look for the name elev
        selected = arcpy.AlterField_management(selected, field.name, 'siteID')"""
selected = arcpy.management.CalculateField(selected, fieldName, expression, "PYTHON3", 
                                codeblock)

deleting  C:/Workspace/VTWetlands/VTWetlands/outputs/dissolved_randompoints.shp
['OBJECTID', 'Shape', 'CID', 'Name']
['OBJECTID', 'Shape', 'IDN', 'ID', 'lat', 'lon', 'sample_dat', 'project', 'Name']


## Extract  Data at Points 

In [46]:
names = ["copy_selected_sites","dissolved_random"]
files = [wd+"VTWetlands.gdb/"+"copy_selected_sites",
         wd+"VTWetlands.gdb/"+"dissolved_random"]
newname="merged_extracted"
print(files)

emptyFC = arcpy.CreateFeatureclass_management(wd+"VTWetlands.gdb",
                                              newname,
                                              "POINT", 
                                              files[0])

merge = arcpy.management.Merge(files,emptyFC, "", "ADD_SOURCE_INFO")

table=arcpy_globExtractMultiValues2Points(inpath="D:/GeoData/Cropscape_VT",
                                    globsearch="*.tif",
                                    inpoints=merge)
table=arcpy_globExtractMultiValues2Points(inpath='D:\\GeoData\\LandLandcov_BaseLC2016\\',
                                    globsearch="*.tif",
                                    inpoints=merge)
table=arcpy_globExtractMultiValues2Points(inpath=outfolder,
                                    globsearch="*.tif",
                                    inpoints=merge)

arcpy.management.CopyFeatures(in_features=table,
                              out_feature_class=outfolder+newname)
    
outpath=r"C:\\Workspace\\VTWetlands\\"
outname = r"merged_extracted.csv"
if os.path.exists(os.path.join(outpath,outname)):
    print(os.path.join(outpath,outname))
    os.remove(os.path.join(outpath,outname))
arcpy.conversion.TableToTable(table,outpath,outname)
    

['C:/Workspace/VTWetlands/VTWetlands/VTWetlands.gdb/copy_selected_sites', 'C:/Workspace/VTWetlands/VTWetlands/VTWetlands.gdb/dissolved_random']
['CDL_2008_clip_20201002155541_287304339.tif', 'CDL_2009_clip_20201002155541_287304339.tif', 'CDL_2010_clip_20201002155541_287304339.tif', 'CDL_2011_clip_20201002155541_287304339.tif', 'CDL_2012_clip_20201002155541_287304339.tif', 'CDL_2013_clip_20201002155541_287304339.tif', 'CDL_2014_clip_20201002155541_287304339.tif', 'CDL_2015_clip_20201002155541_287304339.tif', 'CDL_2016_clip_20201002155541_287304339.tif', 'CDL_2017_clip_20201002155541_287304339.tif', 'CDL_2018_clip_20201002155541_287304339.tif', 'CDL_2019_clip_20201002155541_287304339.tif']
['LandLandcov_BaseLC2016.tif']
['lclu16_agr.tif', 'lclu16_imperv.tif', 'lclu16_shr.tif', 'lclu16_wet.tif', 'rcpp_score.tif', 'vswi_advis.tif', 'vswi_class.tif', 'vtss_hygr.tif']
C:\\Workspace\\VTWetlands\\merged_extracted.csv


<Result 'C:\\Workspace\\VTWetlands\\merged_extracted.csv'>

## Randomly asign points to each student

In [47]:
tempdir = wd + "temp/"
#print(tempdir)
tempfiles = os.listdir(tempdir)
#print(tempfiles)
if os.path.exists(tempdir):
    try:
        os.remove(tempdir)
        os.mkdir(tempdir)
    except:
        "unable to delete tempdir"
        for file in tempfiles:
            filepath = os.path.join(tempdir,file)
            try:
                os.remove(filepath)
            except:
                print("unable to delete",filepath)


fdir = wd + "outputs/kmls/"
for file in tempfiles:
    filepath = os.path.join(fdir,file)
    try:
        os.remove(filepath)
    except:
        print("unable to delete",filepath)


# set starting parameters for loop
IDmin_sample = 0
IDmin_rand = 0
start = 0
stop = len(df.IDN)
arr = np.arange(start,stop)
print(arr)
netid_shuff = np.array(netid[:nstudents])
np.random.shuffle(netid_shuff)
print(netid_shuff)

unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/acady.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/acady.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/acady.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/acady.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/acady.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/acady.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/acady.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/acady.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/aesherma.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/aesherma.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/aesherma.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/aesherma.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/aesherma.sbx
unable

unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/ciobrien.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/ciobrien.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/ciobrien.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/ciobrien.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/ciobrien.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/ciobrien.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/ciobrien.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/cmconway.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/cmconway.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/cmconway.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/cmconway.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/cmconway.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/

unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/eschnei1.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/eschnei1.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/eschnei1.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/eschnei1.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/eschnei1.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/eschnei1.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/extra1.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/extra1.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/extra1.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/extra1.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/extra1.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/extra1.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/extra1.

unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkehmna.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkehmna.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkehmna.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkehmna.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkuhl.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkuhl.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkuhl.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkuhl.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkuhl.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkuhl.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkuhl.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkuhl.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jlane4.cpg
unable t

unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/kmillman.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/kmillman.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/kmillman.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/lbrown28.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/lbrown28.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/lbrown28.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/lbrown28.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/lbrown28.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/lbrown28.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/lbrown28.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/lbrown28.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/mczimmer.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outp

...

unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_emwong.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_emwong.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_emwong.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_epepe.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_epepe.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_epepe.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_epepe.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_epepe.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_epepe.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_epepe.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points2_sample_epepe.shx

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_mswartl1.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_mswartl1.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_mswartl1.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_mswartl1.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_mswartl1.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_mswartl1.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_nebingha.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_nebingha.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_nebingha.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_nebingha.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_neb

unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_sralger.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_sralger.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_sralger.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_sralger.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_sralger.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_szigic.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_szigic.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_szigic.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_szigic.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_szigic.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_random_s

unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_awpierce.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_awpierce.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_awpierce.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_awpierce.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_awpierce.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_bbalpard.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_bbalpard.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_bbalpard.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_bbalpard.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points3_sample_bbalpard.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/p

In [48]:
for i in range(nreps):
    # shuffle netids to randomize plot assignments
    np.random.shuffle(netid_shuff)
    
    # RANDOM
    print ("iteration:",i)
    print ("IDmin_rand:",IDmin_rand)
    print ("IDmin_sample:",IDmin_sample)
    inlayer = random
    nstudents= len(roster[:nstudents])
    z = .75 # the proportion of nonoverlap, 
    npoints= 4
    col = "OBJECTID"
    
    IDmin_rand = random_selection1(inlayer,nstudents,IDmin_rand,z,npoints,col,netid_shuff,i)
    
    # SELECTED
    inlayer = selected
    np.random.shuffle(arr)
    # parameters
    nstudents= len(roster[:nstudents])
    z = 0 # the proportion of nonoverlap
    npoints= 1
    nstudentsperpoint = 9
    col = "OBJECTID"
    IDmin_sample = sample_selection2(inlayer,nstudents,IDmin_sample,z,
                                     npoints,col,netid_shuff,i,arr,nstudentsperpoint)



fpath = wd + "temp/"
for nid in netid[:nstudents]:
    print(nid)
    files = globsearchinpath(fpath,"points*"+nid+"*shp")
    
    # Process:  Create a new empty feature class to append shapefiles to
    print(files)
    
    
    emptyFC = arcpy.CreateFeatureclass_management(wd+"temp/",
                                              nid,
                                              "POINT", 
                                              files[0])
    merge = arcpy.management.Merge(files,emptyFC, "", "ADD_SOURCE_INFO")

    #copy = arcpy.management.CopyFeatures(merge, wd+"VTWetlands.gdb/"+outname)
    #arcpy.conversion.LayerToKML(layer=merge,
    #                            out_kmz_file=outfolder+"kmls/"+nid+".kmz")
    shppath=wd+"temp/"+nid+".shp"
    kmlpath=outfolder+"kmls/"+nid+".kml"

    os_cmd_ogr2ogr(outpath=kmlpath,inpath=shppath,outformat="KML")
    

index 1
student epepe
3 7
OBJECTID > 3  And OBJECTID <= 7
index 2
student dkager
6 10
OBJECTID > 6  And OBJECTID <= 10
index 3
student emwong
9 13
OBJECTID > 9  And OBJECTID <= 13
index 4
student nsteinth
12 16
OBJECTID > 12  And OBJECTID <= 16
index 5
student eroy4
15 19
OBJECTID > 15  And OBJECTID <= 19
index 6
student dullatho
18 22
OBJECTID > 18  And OBJECTID <= 22
index 7
student jkehmna
21 25
OBJECTID > 21  And OBJECTID <= 25
index 8
student kkoberna
24 28
OBJECTID > 24  And OBJECTID <= 28
index 9
student ntryan
27 31
OBJECTID > 27  And OBJECTID <= 31
index 10
student cwalther
30 34
OBJECTID > 30  And OBJECTID <= 34
index 11
student kchealy
33 37
OBJECTID > 33  And OBJECTID <= 37
index 12
student nebingha
36 40
OBJECTID > 36  And OBJECTID <= 40
index 13
student msetzke
39 43
OBJECTID > 39  And OBJECTID <= 43
index 14
student kmillman
42 46
OBJECTID > 42  And OBJECTID <= 46
index 15
student jgess
45 49
OBJECTID > 45  And OBJECTID <= 49
index 16
student jdely
48 52
OBJECTID > 48  A

index 58
student wkowalch
6 [68]
OBJECTID = 68
index 59
student lbrown28
6 [68]
OBJECTID = 68
index 60
student ciobrien
6 [68]
OBJECTID = 68
index 61
student jgoodma4
6 [68]
OBJECTID = 68
index 62
student eredmore
6 [68]
OBJECTID = 68
index 63
student msanange
7 [43]
OBJECTID = 43
index 64
student jlane4
7 [43]
OBJECTID = 43
index 65
student ostern
7 [43]
OBJECTID = 43
index 66
student jspindel
7 [43]
OBJECTID = 43
index 67
student agausepo
7 [43]
OBJECTID = 43
index 68
student awiegman
7 [43]
OBJECTID = 43
index 69
student extra5
7 [43]
OBJECTID = 43
index 70
student eschnei1
7 [43]
OBJECTID = 43
index 71
student mdisaia
7 [43]
OBJECTID = 43
index 72
student mfriedb1
8 [18]
OBJECTID = 18
index 73
student elparks
8 [18]
OBJECTID = 18
index 74
student bgroppe
8 [18]
OBJECTID = 18
index 75
student dncampbe
8 [18]
OBJECTID = 18
index 76
student cmconway
8 [18]
OBJECTID = 18
iteration: 1
IDmin_rand: 228
IDmin_sample: 9
index 0
student jlane4
228 232
OBJECTID > 228  And OBJECTID <= 232
inde

student dncampbe
3 [10]
OBJECTID = 10
index 33
student jspindel
3 [10]
OBJECTID = 10
index 34
student mczimmer
3 [10]
OBJECTID = 10
index 35
student msanange
3 [10]
OBJECTID = 10
index 36
student dkager
4 [48]
OBJECTID = 48
index 37
student camcclel
4 [48]
OBJECTID = 48
index 38
student gmccarth
4 [48]
OBJECTID = 48
index 39
student rsleo
4 [48]
OBJECTID = 48
index 40
student epetters
4 [48]
OBJECTID = 48
index 41
student spurdy
4 [48]
OBJECTID = 48
index 42
student mfriedb1
4 [48]
OBJECTID = 48
index 43
student extra2
4 [48]
OBJECTID = 48
index 44
student amhildeb
4 [48]
OBJECTID = 48
index 45
student szigic
5 [26]
OBJECTID = 26
index 46
student kmillman
5 [26]
OBJECTID = 26
index 47
student jlpetty
5 [26]
OBJECTID = 26
index 48
student dullatho
5 [26]
OBJECTID = 26
index 49
student kkoberna
5 [26]
OBJECTID = 26
index 50
student eschnei1
5 [26]
OBJECTID = 26
index 51
student eroy4
5 [26]
OBJECTID = 26
index 52
student pheaton
5 [26]
OBJECTID = 26
index 53
student extra1
5 [26]
OBJECTI

index 7
student jkehmna
0 [53]
OBJECTID = 53
index 8
student spurdy
0 [53]
OBJECTID = 53
index 9
student ostern
1 [30]
OBJECTID = 30
index 10
student extra3
1 [30]
OBJECTID = 30
index 11
student ciobrien
1 [30]
OBJECTID = 30
index 12
student awpierce
1 [30]
OBJECTID = 30
index 13
student jpastore
1 [30]
OBJECTID = 30
index 14
student msanange
1 [30]
OBJECTID = 30
index 15
student jkuhl
1 [30]
OBJECTID = 30
index 16
student rwatt
1 [30]
OBJECTID = 30
index 17
student eredmore
1 [30]
OBJECTID = 30
index 18
student kiriker
2 [32]
OBJECTID = 32
index 19
student mkovalci
2 [32]
OBJECTID = 32
index 20
student amhildeb
2 [32]
OBJECTID = 32
index 21
student jspindel
2 [32]
OBJECTID = 32
index 22
student mswartl1
2 [32]
OBJECTID = 32
index 23
student agausepo
2 [32]
OBJECTID = 32
index 24
student dullatho
2 [32]
OBJECTID = 32
index 25
student jlane4
2 [32]
OBJECTID = 32
index 26
student jdely
2 [32]
OBJECTID = 32
index 27
student kchealy
3 [69]
OBJECTID = 69
index 28
student dkager
3 [69]
OBJEC

OBJECTID > 897  And OBJECTID <= 901
index 72
student hnovembe
900 904
OBJECTID > 900  And OBJECTID <= 904
index 73
student cwalther
903 907
OBJECTID > 903  And OBJECTID <= 907
index 74
student epepe
906 910
OBJECTID > 906  And OBJECTID <= 910
index 75
student agausepo
909 913
OBJECTID > 909  And OBJECTID <= 913
index 76
student kchealy
912 916
OBJECTID > 912  And OBJECTID <= 916
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
index 0
student lbrown28
0 [64]
OBJECTID = 64
index 1
student agigler
0 [64]
OBJECTID = 64
index 2
student jlmashri
0 [64]
OBJECTID = 64
index 3
student rw

...

['points0_Asample_jdely.shp', 'points0_Brandom_jdely.shp', 'points1_Asample_jdely.shp', 'points1_Brandom_jdely.shp', 'points2_Asample_jdely.shp', 'points2_Brandom_jdely.shp', 'points3_Asample_jdely.shp', 'points3_Brandom_jdely.shp', 'points4_Asample_jdely.shp', 'points4_Brandom_jdely.shp', 'points5_Asample_jdely.shp', 'points5_Brandom_jdely.shp', 'points6_Asample_jdely.shp', 'points6_Brandom_jdely.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jdely.kml C:/Workspace/VTWetlands/VTWetlands/temp/jdely.shp
success
mfriedb1
['points0_Asample_mfriedb1.shp', 'points0_Brandom_mfriedb1.shp', 'points1_Asample_mfriedb1.shp', 'points1_Brandom_mfriedb1.shp', 'points2_Asample_mfriedb1.shp', 'points2_Brandom_mfriedb1.shp', 'points3_Asample_mfriedb1.shp', 'points3_Brandom_mfriedb1.shp', 'points4_Asample_mfriedb1.shp', 'points4_Brandom_mfriedb1.shp', 'points5_Asample_mfriedb1.shp', 'points5_Brandom_mfriedb1.shp', 'points6_Asample_mfriedb1.shp', 'points6_Brandom_mfriedb1.shp']
og

['points0_Asample_jkehmna.shp', 'points0_Brandom_jkehmna.shp', 'points1_Asample_jkehmna.shp', 'points1_Brandom_jkehmna.shp', 'points2_Asample_jkehmna.shp', 'points2_Brandom_jkehmna.shp', 'points3_Asample_jkehmna.shp', 'points3_Brandom_jkehmna.shp', 'points4_Asample_jkehmna.shp', 'points4_Brandom_jkehmna.shp', 'points5_Asample_jkehmna.shp', 'points5_Brandom_jkehmna.shp', 'points6_Asample_jkehmna.shp', 'points6_Brandom_jkehmna.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkehmna.kml C:/Workspace/VTWetlands/VTWetlands/temp/jkehmna.shp
success
kkoberna
['points0_Asample_kkoberna.shp', 'points0_Brandom_kkoberna.shp', 'points1_Asample_kkoberna.shp', 'points1_Brandom_kkoberna.shp', 'points2_Asample_kkoberna.shp', 'points2_Brandom_kkoberna.shp', 'points3_Asample_kkoberna.shp', 'points3_Brandom_kkoberna.shp', 'points4_Asample_kkoberna.shp', 'points4_Brandom_kkoberna.shp', 'points5_Asample_kkoberna.shp', 'points5_Brandom_kkoberna.shp', 'points6_Asample_kkoberna.shp', 'p

['points0_Asample_amorton2.shp', 'points0_Brandom_amorton2.shp', 'points1_Asample_amorton2.shp', 'points1_Brandom_amorton2.shp', 'points2_Asample_amorton2.shp', 'points2_Brandom_amorton2.shp', 'points3_Asample_amorton2.shp', 'points3_Brandom_amorton2.shp', 'points4_Asample_amorton2.shp', 'points4_Brandom_amorton2.shp', 'points5_Asample_amorton2.shp', 'points5_Brandom_amorton2.shp', 'points6_Asample_amorton2.shp', 'points6_Brandom_amorton2.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/amorton2.kml C:/Workspace/VTWetlands/VTWetlands/temp/amorton2.shp
success
hnovembe
['points0_Asample_hnovembe.shp', 'points0_Brandom_hnovembe.shp', 'points1_Asample_hnovembe.shp', 'points1_Brandom_hnovembe.shp', 'points2_Asample_hnovembe.shp', 'points2_Brandom_hnovembe.shp', 'points3_Asample_hnovembe.shp', 'points3_Brandom_hnovembe.shp', 'points4_Asample_hnovembe.shp', 'points4_Brandom_hnovembe.shp', 'points5_Asample_hnovembe.shp', 'points5_Brandom_hnovembe.shp', 'points6_Asample_h

['points0_Asample_msanange.shp', 'points0_Brandom_msanange.shp', 'points1_Asample_msanange.shp', 'points1_Brandom_msanange.shp', 'points2_Asample_msanange.shp', 'points2_Brandom_msanange.shp', 'points3_Asample_msanange.shp', 'points3_Brandom_msanange.shp', 'points4_Asample_msanange.shp', 'points4_Brandom_msanange.shp', 'points5_Asample_msanange.shp', 'points5_Brandom_msanange.shp', 'points6_Asample_msanange.shp', 'points6_Brandom_msanange.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/msanange.kml C:/Workspace/VTWetlands/VTWetlands/temp/msanange.shp
success
eschnei1
['points0_Asample_eschnei1.shp', 'points0_Brandom_eschnei1.shp', 'points1_Asample_eschnei1.shp', 'points1_Brandom_eschnei1.shp', 'points2_Asample_eschnei1.shp', 'points2_Brandom_eschnei1.shp', 'points3_Asample_eschnei1.shp', 'points3_Brandom_eschnei1.shp', 'points4_Asample_eschnei1.shp', 'points4_Brandom_eschnei1.shp', 'points5_Asample_eschnei1.shp', 'points5_Brandom_eschnei1.shp', 'points6_Asample_e

ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/mczimmer.kml C:/Workspace/VTWetlands/VTWetlands/temp/mczimmer.shp
success
eroy4
['points0_Asample_eroy4.shp', 'points0_Brandom_eroy4.shp', 'points1_Asample_eroy4.shp', 'points1_Brandom_eroy4.shp', 'points2_Asample_eroy4.shp', 'points2_Brandom_eroy4.shp', 'points3_Asample_eroy4.shp', 'points3_Brandom_eroy4.shp', 'points4_Asample_eroy4.shp', 'points4_Brandom_eroy4.shp', 'points5_Asample_eroy4.shp', 'points5_Brandom_eroy4.shp', 'points6_Asample_eroy4.shp', 'points6_Brandom_eroy4.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/eroy4.kml C:/Workspace/VTWetlands/VTWetlands/temp/eroy4.shp
success
awiegman
['points0_Asample_awiegman.shp', 'points0_Brandom_awiegman.shp', 'points1_Asample_awiegman.shp', 'points1_Brandom_awiegman.shp', 'points2_Asample_awiegman.shp', 'points2_Brandom_awiegman.shp', 'points3_Asample_awiegman.shp', 'points3_Brandom_awiegman.shp', 'points4_Asample_awiegman.shp', 'points4_Brandom_

In [49]:
try:
    os.remove(tempdir)
    os.mkdir(tempdir)
except:
    print("unable to delete tempdir")

unable to delete tempdir
